# <center id="p1">  🎚 Подбор идеальных гиперпараметров с `ClearML`!</center>

<div class="alert alert-info">

<img src="../images/hpo.webp" align="center" height=500>
<br>

`ClearML` позволяет запустить подбор гипрепараметров любого эксперимента, зная только его `ID`, написав минимум дополнительного кода.

* Просто выставляем интервалы для перебора гиперпараметров
* Запускаем "демона" `ClearML-Agent`
* Передаём ему количество итераций перебора, которое хотим провести, и `ID`
* Занимаемся своими делами и спокойно мониторим в `WebUI` прогресс перебора.

<div class="alert alert-info">
    
Для начала подбора идеальных гиперпараметров нашего эксперимента понадобятся 2 вещи:

1. Установить и запустить `clearml-agent daemon` в консоли командой `clearml-agent daemon --queue default`.
2. Перед запуском демона следует в консоли ввести команду `clearml-init` и заполнить ключи доступа, которые сохранятся в файле `clearml.conf` и при следующих запусках их не потребуется вводить.

In [1]:
# установим необходимые библиотеки
! pip install -U clearml-agent optuna clearml lightning -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.1/443.1 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 818.9/818.9 kB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 74.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8

In [ ]:
# Запускаем clearml-init, и вводим все credentials
!clearml-init

<div class="alert alert-info">

Запустим скрипт, который создали в предыдущем ноутбуке.

In [ ]:
!python train_script.py

ClearML Task: created new task id=179ac345a3c04f8e955efed963f5e944
2025-04-14 18:53:14,966 - clearml.Task - INFO - No repository found, storing script code instead
2025-04-14 18:53:17.602861: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744656797.865326    1864 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744656797.937343    1864 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-14 18:53:18.492020: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild T

<div class="alert alert-success">

Эксперимент отработал до конца без ошибок.

Удобно, что мы в конце вывели `Task ID` сразу вставим его в ячейку ниже.

In [3]:
########################################################
### Не забудьте заменить на свой собственный task id ###
########################################################
TEMPLATE_TASK_ID = "179ac345a3c04f8e955efed963f5e944"

# <center> 👹 Запускаем демона - ClearML Agent

<div class="alert alert-info">

**Агент ClearML** — это исполнительный компонент (демон) платформы ClearML. Его основная задача — удаленно запускать и управлять выполнением задач (экспериментов), которые были поставлены в очередь на сервере ClearML.

<div class="alert alert-info">

Теперь запустим "демон" `ClearML Agent` отдельным процессом в фоне

* указываем название очереди в парметре `--queue` (в нашем случае `default`)
* при необходимости добавляем параметр `--detached` (или символ `&`), чтобы процесс висел в фоне и не завершился при закрытии консоли.

In [5]:
!clearml-agent daemon --queue default --detached


clearml_agent: ERROR: Instance with the same WORKER_ID [204e54f43af8:gpuall] is already running



# <center> 🚀 Начинаем подбор параметров 🎛

<div class="alert alert-info">

Возьмём хорошо известный `Optuna Optimizer`, для которого есть интеграция в `ClearML`. Так же поддерживаются пребор по сетке и `Randomized Search`.

In [6]:
from clearml.automation import UniformParameterRange, UniformIntegerParameterRange
from clearml.automation import HyperParameterOptimizer
from clearml.automation.optuna import OptimizerOptuna
from clearml import Task

In [7]:
# Запускаем новый Task c оптимизацией
task = Task.init(
    project_name="Hyperparameter Optimization with Optuna",
    task_name="Hyperparameter Search",
    task_type=Task.TaskTypes.optimizer,
)

ClearML Task: created new task id=76afc56ffe5843a89e29b670366f5d2d
2025-04-21 21:30:16,538 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/404d1633cd1d4033b49adcbe0299c12e/experiments/76afc56ffe5843a89e29b670366f5d2d/output/log


<div class="alert alert-info">

**Возможные интервалы значений для перебора:**
* `UniformParameterRange` - значения выбираются равномерно из заданного диапазона
* `LogUniformParameterRange` - параметр может принимать значения, отличающиеся порядками, и важно равномерно исследовать их логарифмы
* `ChoiceParameterRange` - значения выбираются из заранее определенного списка вариантов
* `DiscreteParameterRange` - значения выбираются из набора заранее определённых дискретных значений, не распределённых равномерно по непрерывному диапазону.
* `UniformIntegerParameterRange` - выбираются целочисленные значения из заданного распределения

<div class="alert alert-info">

**Так же важно указать:**
* `ID` эксперимента
* Метрику и направление в которое хотим прийти (уменьшить или увеличить)
* Название очереди "демона" (в нашем случае `default`)
* Гиперпараметры передаем через префикс `General/`, который берём на вкладке `CONFIGURATION` базового эксперимента.

In [8]:
optimizer = HyperParameterOptimizer(
    base_task_id=TEMPLATE_TASK_ID,  # ID эксперимента

    # список гиперпараметров для оптимизации
    hyper_parameters=[
        UniformIntegerParameterRange(
            "General/num_epochs", min_value=2, max_value=5, step_size=2
        ),
        UniformIntegerParameterRange(
            "General/batch_size", min_value=16, max_value=24, step_size=2
        ),
        UniformParameterRange(
            "General/dropout", min_value=0, max_value=0.5, step_size=0.05
        ),
        UniformParameterRange(
            "General/learning_rate",
            min_value=0.00025,
            max_value=0.01,
            step_size=0.00025,
        ),
    ],
    # метрика, которую мы хотим увеличить или уменьшить
    objective_metric_title="test_accuracy",
    objective_metric_series="test_accuracy",
    objective_metric_sign="max",  # maximize or minimize
    # выбираем оптимайзер - СlearML поддерживает GridSearch, RandomSearch, OptimizerBOHB and OptimizerOptuna
    optimizer_class=OptimizerOptuna,

    # Настраиваем оптимизатор
    execution_queue="default",  # название очереди
    max_number_of_concurrent_tasks=1,  # количество параллельных запусков
    optimization_time_limit=60.0,  # временной лимит для подбора
    compute_time_limit=120,  # временной лимит для всех вычислений (сумма на всех машинах)
    total_max_jobs=5,  # количество экспериментов оптимизации
    # Минимальное и максимальное число итераций на эксперимент
    min_iteration_per_job=15000,
    max_iteration_per_job=1500000,
)

In [9]:
optimizer.set_report_period(1)  # устанавливаем интервал для появления логов
optimizer.start() # запускаем оптимизацию
optimizer.wait()  # ждём пока процесс завершится
optimizer.stop()  # отключаем все оптимизационные таски, даже запущенные в бэкграунде

[I 2025-04-21 21:35:23,706] A new study created in memory with name: 76afc56ffe5843a89e29b670366f5d2d
/usr/local/lib/python3.11/dist-packages/optuna/distributions.py:699: UserWarning:

The distribution is specified by [2, 5] and step=2, but the range is not divisible by `step`. It will be replaced by [2, 4].

/usr/local/lib/python3.11/dist-packages/clearml/automation/optuna/optuna.py:41: FutureWarning:

suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., step=...) instead.



Progress report #0 completed, sleeping for 0.25 minutes
2025-04-21 21:35:31,902 - clearml.automation.optimization - INFO - Creating new Task: {'General/num_epochs': 2, 'General/batch_size': 24, 'General/dropout': 0.30000000000000004, 'General/learning_rate': 0.007500000000000001}
Progress report #1 completed, sleeping for 1.0 minutes
Progress report #2 completed, sleeping for 1.0 minutes
Progress report #3 completed, sleeping for 1.0 minutes
Progress report #4 completed, sleeping for 1.0 minutes
Progress report #5 completed, sleeping for 1.0 minutes
Progress report #6 completed, sleeping for 1.0 minutes
Progress report #7 completed, sleeping for 1.0 minutes
Progress report #8 completed, sleeping for 1.0 minutes
Progress report #9 completed, sleeping for 1.0 minutes


/usr/local/lib/python3.11/dist-packages/optuna/trial/_trial.py:497: UserWarning:

The reported value is ignored because this `step` 4168 is already reported.

[I 2025-04-21 21:44:18,273] Trial 0 finished with value: 0.5267784 and parameters: {'General/num_epochs': 2, 'General/batch_size': 24, 'General/dropout': 0.30000000000000004, 'General/learning_rate': 0.007500000000000001}. Best is trial 0 with value: 0.5267784.


OptunaObjective result metric=0.5267784, iteration (4168, 0.5267783999443054)
2025-04-21 21:44:25,538 - clearml.automation.optimization - INFO - Creating new Task: {'General/num_epochs': 4, 'General/batch_size': 24, 'General/dropout': 0.4, 'General/learning_rate': 0.0022500000000000003}
Updating job performance summary plot/table
Progress report #10 completed, sleeping for 1.0 minutes
Progress report #11 completed, sleeping for 1.0 minutes
Progress report #12 completed, sleeping for 1.0 minutes
Progress report #13 completed, sleeping for 1.0 minutes
Progress report #14 completed, sleeping for 1.0 minutes
Progress report #15 completed, sleeping for 1.0 minutes


/usr/local/lib/python3.11/dist-packages/optuna/trial/_trial.py:497: UserWarning:

The reported value is ignored because this `step` 8336 is already reported.

[I 2025-04-21 21:50:48,839] Trial 1 finished with value: 0.51928467 and parameters: {'General/num_epochs': 4, 'General/batch_size': 24, 'General/dropout': 0.4, 'General/learning_rate': 0.0022500000000000003}. Best is trial 0 with value: 0.5267784.


...

OptunaObjective result metric=0.4901576, iteration (8336, 0.4901576042175293)
2025-04-21 21:57:40,656 - clearml.automation.optimization - INFO - Creating new Task: {'General/num_epochs': 4, 'General/batch_size': 18, 'General/dropout': 0.2, 'General/learning_rate': 0.0035000000000000005}
Updating job performance summary plot/table
Progress report #22 completed, sleeping for 1.0 minutes
Progress report #23 completed, sleeping for 1.0 minutes
Progress report #24 completed, sleeping for 1.0 minutes
Progress report #25 completed, sleeping for 1.0 minutes
Progress report #26 completed, sleeping for 1.0 minutes
Progress report #27 completed, sleeping for 1.0 minutes


/usr/local/lib/python3.11/dist-packages/optuna/trial/_trial.py:497: UserWarning:

The reported value is ignored because this `step` 11112 is already reported.

[I 2025-04-21 22:04:22,945] Trial 3 finished with value: 0.56564766 and parameters: {'General/num_epochs': 4, 'General/batch_size': 18, 'General/dropout': 0.2, 'General/learning_rate': 0.0035000000000000005}. Best is trial 3 with value: 0.56564766.


OptunaObjective result metric=0.56564766, iteration (11112, 0.5656476616859436)
2025-04-21 22:04:30,213 - clearml.automation.optimization - INFO - Creating new Task: {'General/num_epochs': 2, 'General/batch_size': 20, 'General/dropout': 0.5, 'General/learning_rate': 0.006500000000000001}
Updating job performance summary plot/table
Progress report #28 completed, sleeping for 1.0 minutes
Progress report #29 completed, sleeping for 1.0 minutes
Progress report #30 completed, sleeping for 1.0 minutes
Progress report #31 completed, sleeping for 1.0 minutes
Progress report #32 completed, sleeping for 1.0 minutes


/usr/local/lib/python3.11/dist-packages/optuna/trial/_trial.py:497: UserWarning:

The reported value is ignored because this `step` 5000 is already reported.

[I 2025-04-21 22:09:50,616] Trial 4 finished with value: 0.49710009 and parameters: {'General/num_epochs': 2, 'General/batch_size': 20, 'General/dropout': 0.5, 'General/learning_rate': 0.006500000000000001}. Best is trial 3 with value: 0.56564766.


OptunaObjective result metric=0.49710009, iteration (5000, 0.4971000850200653)
Updating job performance summary plot/table
Updating job performance summary plot/table


# <center> 🏁 Выводим результаты лучшего эксперимента

In [10]:
# Посмотрим гиперпараметры лучшего эксперимента
k = 5
top_exp = optimizer.get_top_experiments(top_k=k)
print("Top {} experiments are:".format(k))
for n, t in enumerate(top_exp, 1):
    print(
        "Rank {}: task id={} |result={}".format(
            n, t.id, t.get_last_scalar_metrics()["test_accuracy"]["test_accuracy"]["last"]
        )
    )

Top 5 experiments are:
Rank 1: task id=dd58f736b599426e92cee2920281a99a |result=0.56564766
Rank 2: task id=eff852e2517045cb931420af149b807f |result=0.5267784
Rank 3: task id=a068b99889c14e7aa0cd8a48a47b2041 |result=0.51928467
Rank 4: task id=bce2b360e5c84c239dd00e734d23be9e |result=0.49710009
Rank 5: task id=c1e67102c03b46f1b0d98a13e87e0716 |result=0.4901576


In [11]:
# Завершаем оптимизационную задачу
task.close()

На вкладке `PLOTS` таска с оптимизацией можно увидеть все проведённые эксперименты, отсортировать по нужному столбцу или перейти по ссылке на страницу нужного эксперимента:

<img src="../images/hpo2.png" align="center" height=500>

# <center> 🧸 Выводы и заключения ✅

<div class="alert alert-success">

* Оптимизация гиперпараметров в **ClearML** очень удобный инструмент, если вы уже пользуетесь **ClearML** для логирования экспериментов.
* Требует минимум кода
* Позволяет запустить этот процесс в фоне и дальше заниматься своими делами на компьютере
* Контролировать процесс и получать результаты подбора через удобный WebUI